Import Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import os
import s3fs
import boto3
from io import BytesIO as bo
import gc

Calculate Date Variables

In [2]:
# Variables
#FOR CUR_PROC_WK, TODAY, PRE_PROC_WK,CUR_WK,PRE_WK
this_day = datetime.today()
# ### REMOVE LATER ###
this_day = this_day - timedelta(days=7)
# ###
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)
CUR_PROC_WK = monday.strftime("%Y%m%d")  #Converting to desired string format
TODAY = monday  #Keeping datetime format
PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)
CAL_RECORD_DATE = (monday + timedelta(days = 1)).strftime("%Y-%m-%d")
PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)


#this is for 16a?
month_end_date = CUR_WK0.replace(day=1) - timedelta(days=1)
if 0 <= (CUR_WK0 - month_end_date).days < 7:
    CUT_OFF = (CUR_WK0.replace(day=1)).date()
else:
    CUT_OFF = (CUR_WK0 + timedelta(days=1)).date()
    
CUT_OFF2 = (CUR_WK0 - timedelta(days=105*7) + timedelta(days=1)).date()


In [3]:
# Grabbing IW Release File 
s3 = s3fs.S3FileSystem()
bucket_list = s3.listdir(f'vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/inbound/')
for file in bucket_list:
    if (file['Key'].__contains__(f"IRWD_RELEASE_WKLY_{CUR_PROC_WK}")):
        release_file = file['Key']

# Reading Release File -
s3_client = boto3.client('s3')
bucket = release_file.split('/')[0]
file_key = release_file.split('/',1)[1]

adm = pd.read_csv(f's3://{bucket}/{file_key}',sep='|')
adm1 = adm[CUR_PROC_WK].tolist()

for item in adm1:
    if item.startswith('P_OUT_110_CALL_ATND'):
        CALL_TIMESTAMP = item.split('_')[-1].split('.')[0]

Library Names and File Paths :

In [4]:
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
callpweek = f'PYADM/weekly/archive/{PRE_WK}/calls_samples/'
calweek = 'PYADM/weekly/staging/calls_samples/'
calmnth = 'PYADM/monthly/staging/calls_samples/'

In [5]:
#For fixing call timestamp bug # GET IT FIXED FROM IW !!!!
CALL_TIMESTAMP = '20240729022312'

---

In [6]:
dtype_dict = {
    3:str,4:'Int64',5:str,6:str,8:str,
    12:str,13:str,15:str,16:str,18:str,21:str,
    22:str,23:str,24:str,25:str,26:str,28:str
}
curwk_CALL = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_CALL_{CALL_TIMESTAMP}.TXT',
    delimiter='|',
    dtype= dtype_dict,
    on_bad_lines='warn',
    quoting=3
)
# Fixing Date Cols - 
curwk_CALL['CallDateTime'] = pd.to_datetime(curwk_CALL['CallDateTime'], format='%b %d %Y %I:%M%p', errors='coerce')
curwk_CALL['StartDate'] = pd.to_datetime(curwk_CALL['StartDate'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
curwk_CALL['EndDate'] = pd.to_datetime(curwk_CALL['StartDate'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

In [7]:
#7 mins for import 

In [8]:
curwk_ATTENDEE = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_CALL-ATND_{CALL_TIMESTAMP}.TXT', delimiter='|',dtype={0:str,'CallID':str},quoting=3)
curwk_DISBURSEMENT = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_CALL-DISB_{CALL_TIMESTAMP}.TXT', delimiter='|',dtype={1:str,'CallID':str},parse_dates=[9],quoting=3)
curwk_CALLPRODUCTDISBURSED = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_CALL-PROD-DISB_{CALL_TIMESTAMP}.TXT', delimiter='|',dtype={'CallID':str},quoting=3)
curwk_PRODUCTDETAIL = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_CALL-ATND-PROD-DTL_{CALL_TIMESTAMP}.TXT', delimiter='|',dtype={2:'Int64','CallID':str},quoting=3)
curwk_CALLDISCUSSION = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_CALL-ATND-DISC_{CALL_TIMESTAMP}.TXT', delimiter='|',dtype={'CallID':str},quoting=3)

In [9]:
# # FOR EXTRA FILE
# curwk_CALLPRODUCTDISBURSED_manual = pd.read_csv(
#     f's3://{bucket}/{raw_path}P_OUT_110_CALL-PROD-DISB_20240614_MANUAL.TXT', 
#     delimiter='|',
#     dtype={'CallID':str,0:str}
# )

# curwk_CALLPRODUCTDISBURSED = pd.concat([curwk_CALLPRODUCTDISBURSED, curwk_CALLPRODUCTDISBURSED_manual], axis=0)

In [10]:
CALL_S = curwk_CALL.drop_duplicates(subset=['CallID'])
ATTENDEE_S = curwk_ATTENDEE.drop_duplicates(subset=['CallID','AttendeeIID'])
DISBURSEMENT_S = curwk_DISBURSEMENT.drop_duplicates(subset=['CallID','PhysicianID','PhysicianTerritoryID','DisbursementID','CallProductDisbursed'])
CALLPRODUCTDISBURSED_S = curwk_CALLPRODUCTDISBURSED.drop_duplicates(subset=['ProductDisbursedProductID','CallProductDescription','CallProductCategory','CallID'])
PRODUCTDETAILS = curwk_PRODUCTDETAIL.drop_duplicates(subset=['CallID','ProductID','ProductPriority'])
CALLDISCUSSIONDETAILS = curwk_CALLDISCUSSION.drop_duplicates(subset=['CallID','CallFocus'])

#### 15- CALLS - Denormalization

In [11]:
CALLS = CALL_S[['CallID','SourceCallID','SalesRepID','SalesRepTerritoryID',
                'CallRecordType','CallType','CallStatus','CallDateTime',
                'CallNotes','DeleteIndicator']].copy()
CALLS = CALLS[CALLS['CallDateTime'] > datetime.strptime('2019-12-31','%Y-%m-%d')]

ATTENDEES = ATTENDEE_S[['CallID','AttendeeIID','AttendeeType']].copy()

CALLS_ATT = pd.merge(CALLS,ATTENDEES,on='CallID',how='left')
CALLS_ATT_DEDUP = CALLS_ATT.drop_duplicates(subset=['CallID','AttendeeIID'])
print("Dupes Dropped : ",(len(CALLS_ATT) - len(CALLS_ATT_DEDUP)))

Dupes Dropped :  0


In [12]:
CALLDISCUSSIONDETAILS_F = CALLDISCUSSIONDETAILS.copy()
CALLDISCUSSIONDETAILS_F['CallFocus_F'] = CALLDISCUSSIONDETAILS_F.groupby('CallID')['CallFocus'].apply(lambda x: ' / '.join(x.dropna().astype(str))).reset_index(drop=True)
CALLDISCUSSIONDETAILS_F.drop_duplicates(subset = ['CallID'], keep='last', inplace = True)

In [13]:
PRODUCTDETAILS_W_DIS = pd.merge(PRODUCTDETAILS,CALLDISCUSSIONDETAILS_F[['CallID','CallFocus_F']],on = 'CallID',how = 'left')

In [14]:
DETAIL_DEDUP = PRODUCTDETAILS_W_DIS.copy()
DETAIL_DEDUP.rename(columns={'CallFocus_F':'CallFocus'},inplace = True)
DETAIL_DEDUP.drop(columns={'ProductDetailID'},inplace= True)
DETAIL_DEDUP.drop_duplicates(subset = ['CallID','ProductID','ProductPriority'],inplace= True)

In [15]:
CALLS_ATT_PR = pd.merge(CALLS_ATT_DEDUP,DETAIL_DEDUP,on = 'CallID',how='left')

#### 15.1: Check all types of product_id

In [16]:
# List of product IDs to exclude
#exclude_ids = ["", "COLOGUARD", "DELZICOL/CANASA", "DUZALLO", "LINZESS", "VIBERZI", "ZURAMPIC"]

# Filter out rows where 'PRODUCTID' is in 'exclude_ids'
#filtered_df = CALLS_ATT_PR[~CALLS_ATT_PR['ProductID'].isin(exclude_ids)]

# Generate frequency table
new_product_call = CALLS_ATT_PR['ProductID'].value_counts().reset_index()

# Rename columns
new_product_call.columns = ['ProductID', 'FREQUENCY']
new_product_call


,ProductID,FREQUENCY
0,LINZESS,11788


In [17]:
frequency_table = PRODUCTDETAILS_W_DIS.groupby(['CallFocus_F', 'ProductID']).size().reset_index(name='FREQUENCY')
frequency_table

,CallFocus_F,ProductID,FREQUENCY
0,,LINZESS,17
1,90 for 30,LINZESS,64
2,90 for 30 / Efficacy – Overall Abdom Symptoms,LINZESS,19
3,Efficacy – Overall Abdom Symptoms,LINZESS,303
4,Efficacy – Overall Abdom Symptoms / 90 for 30,LINZESS,24
5,Efficacy – Overall Abdom Symptoms / Efficacy –...,LINZESS,4
6,Efficacy – Overall Abdom Symptoms / OTC Patien...,LINZESS,4
7,Efficacy – Pain & Constipation,LINZESS,121
8,Efficacy – Pain & Constipation / Efficacy – Ov...,LINZESS,3
9,Efficacy – Pain & Constipation / Formulary,LINZESS,1


In [18]:
DISB = DISBURSEMENT_S[['CallID','PhysicianID','PhysicianTerritoryID','DisbursementID','CallProductDisbursed']].copy()
DISB.rename(columns={'PhysicianID':'AttendeeIID'},inplace=True)
DISB.drop_duplicates(subset=['CallID','AttendeeIID','PhysicianTerritoryID','DisbursementID','CallProductDisbursed'],inplace=True)
DISB_NODUP = DISB.copy()
DISB_NODUP.drop_duplicates(subset = ['DisbursementID','CallID','AttendeeIID','CallProductDisbursed'],inplace=True)

In [19]:
DESCRIPTION_NODUP = CALLPRODUCTDISBURSED_S.copy()
DESCRIPTION_NODUP.drop_duplicates(subset=['CallProductDescription','CallProductCategory','CallID','ProductDisbursedProductID'],inplace=True)

In [20]:
DISB_DESCRIPTION = pd.merge(
    DISB_NODUP,DESCRIPTION_NODUP,
    left_on = ['CallID','CallProductDisbursed'],
    right_on = ['CallID','ProductDisbursedProductID'],
    how = 'left')

In [21]:
exclude_descriptions = ['DUZALLO', 'ZURAMPIC', 'DELZICOL', 'CANASA', 'AHP', 'AHP/GIV', 'VIBERZI', 'GIV']
include_descriptions = ['LINZESS', 'LIN', '145 mcg', '145 mcg Sample 30ct', '290 mcg', '72 mcg', '90 Day Prescription Sell Sheet']
DISB_LIN = DISB_DESCRIPTION.copy()
DISB_LIN['CallProductDescription'] = DISB_LIN['CallProductDescription'].fillna('')
DISB_LIN = DISB_LIN[~DISB_LIN['CallProductDescription'].str.contains('|'.join(exclude_descriptions), case=False)]
DISB_LIN = DISB_LIN[DISB_LIN['CallProductDescription'].str.contains('|'.join(include_descriptions), case=False)]

In [22]:
import re

exclude_descriptions = [re.escape(desc) for desc in['Combined LINZESS/VIBERZI Slim Jim(LINZESS)', 'LINZESS/VIBERZI Bristol Stool Scale Poster (10 Pack)']]
include_descriptions = [re.escape(desc) for desc in['VIBERZI','Large Print PI Cover (ELX32221)', 'Annotated PI (ELX32140)', 'Federal and State Guidance Prescribing (ELX38753)',
                       'Combined VIBERZI/LINZESS Slim Jim(VIBERZI)', 'VIBERZI/LINZESS Bristol Stool Scale Poster (10 Pack)']]
DISB_VIB = DISB_DESCRIPTION.copy()
DISB_VIB['CallProductDescription'] = DISB_VIB['CallProductDescription'].fillna('')
DISB_VIB = DISB_VIB[DISB_VIB['CallProductDescription'].str.contains('|'.join(include_descriptions), case=False)]
DISB_VIB = DISB_VIB[~DISB_VIB['CallProductDescription'].str.contains('|'.join(exclude_descriptions), case=False)]
DISB_VIB.drop(columns={'CallProductDescription'},inplace=True)

In [23]:
DISB_ZUR = DISB_DESCRIPTION.copy()
DISB_ZUR['CallProductDescription'] = DISB_ZUR['CallProductDescription'].fillna('')
DISB_ZUR = DISB_ZUR[DISB_ZUR['CallProductDescription'].str.contains('|'.join(['ZURAMPIC']), case=False)]
DISB_ZUR.drop(columns={'CallProductDescription'},inplace=True)

DISB_DEC = DISB_DESCRIPTION.copy()
DISB_DEC['CallProductDescription'] = DISB_DEC['CallProductDescription'].fillna('')
DISB_DEC = DISB_DEC[DISB_DEC['CallProductDescription'].str.contains('|'.join(['DELZICOL','CANASA']), case=False)]
DISB_DEC.drop(columns={'CallProductDescription'},inplace=True)

DISB_DUZ = DISB_DESCRIPTION.copy()
DISB_DUZ['CallProductDescription'] = DISB_DUZ['CallProductDescription'].fillna('')
DISB_DUZ = DISB_DUZ[DISB_DUZ['CallProductDescription'].str.contains('|'.join(['DUZALLO']), case=False)]
DISB_DUZ.drop(columns={'CallProductDescription'},inplace=True)

DISB_AHP = DISB_DESCRIPTION.copy()
DISB_AHP['CallProductDescription'] = DISB_AHP['CallProductDescription'].fillna('')
DISB_AHP = DISB_AHP[DISB_AHP['CallProductDescription'].str.contains('|'.join(['AHP']), case=False)]
DISB_AHP.drop(columns={'CallProductDescription'},inplace=True)

DISB_AHP_GIV = DISB_DESCRIPTION.copy()
DISB_AHP_GIV['CallProductDescription'] = DISB_AHP_GIV['CallProductDescription'].fillna('')
DISB_AHP_GIV = DISB_AHP_GIV[DISB_AHP_GIV['CallProductDescription'].str.contains('|'.join(['AHP/GIV']), case=False)]
DISB_AHP_GIV.drop(columns={'CallProductDescription'},inplace=True)

In [24]:
CALLS_ATT_PR_LIN= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "LINZESS"].copy()
CALLS_ATT_PR_AHP= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "AHP"].copy()
CALLS_ATT_PR_AHP_GIV= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "AHP/GIV"].copy()
CALLS_ATT_PR_CLG= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "COLOGUARD"].copy()
CALLS_ATT_PR_VIB= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "VIBERZI"].copy()
CALLS_ATT_PR_ZUR= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "ZURAMPIC"].copy()
CALLS_ATT_PR_DEC= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "DELZICOL/CANASA"].copy()
CALLS_ATT_PR_DUZ= CALLS_ATT_PR[CALLS_ATT_PR['ProductID'] == "DUZALLO"].copy()

In [25]:
CALLS_ATT_PR_DISB_LIN = pd.merge(CALLS_ATT_PR_LIN,DISB_LIN,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_AHP = pd.merge(CALLS_ATT_PR_AHP,DISB_AHP,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_AHP_GIV = pd.merge(CALLS_ATT_PR_AHP_GIV,DISB_AHP_GIV,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_VIB = pd.merge(CALLS_ATT_PR_VIB,DISB_VIB,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_ZUR = pd.merge(CALLS_ATT_PR_ZUR,DISB_ZUR,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_DEC = pd.merge(CALLS_ATT_PR_DEC,DISB_DEC,on=['CallID','AttendeeIID'],how='left')
CALLS_ATT_PR_DISB_DUZ = pd.merge(CALLS_ATT_PR_DUZ,DISB_DUZ,on=['CallID','AttendeeIID'],how='left')

In [26]:
CALLS_ATT_PR_DISB = pd.concat([CALLS_ATT_PR_DISB_LIN,CALLS_ATT_PR_DISB_AHP,CALLS_ATT_PR_DISB_AHP_GIV,CALLS_ATT_PR_DISB_VIB,
                              CALLS_ATT_PR_DISB_ZUR,CALLS_ATT_PR_DISB_DEC,CALLS_ATT_PR_DISB_DUZ],ignore_index = True)

In [27]:
PRODUCT = CALLPRODUCTDISBURSED_S.copy()
PRODUCT.drop(columns={'CallProductBatchLotNumber'},inplace=True)
PRODUCT.drop_duplicates(subset=['CallID','ProductDisbursedProductID'],inplace=True)

CALLS_ATT_PR_DISB1 = CALLS_ATT_PR_DISB.sort_values(by='DisbursementID')

#### 15.2: Print all combination: CallRecordType*CallType*ProductID*ProductPriority

In [28]:
frequency_table = CALLS_ATT_PR_DISB1.groupby(['CallRecordType', 'CallType','ProductID','ProductPriority']).size().reset_index(name='FREQUENCY')
frequency_table

,CallRecordType,CallType,ProductID,ProductPriority,FREQUENCY
0,ABBV ENGAGE MEETING,Detail Only,LINZESS,1,100
1,ABBV ENGAGE MEETING,Detail Only,LINZESS,2,27
2,FRX Sales Call,Detail Only,LINZESS,1,3343
3,FRX Sales Call,Detail Only,LINZESS,2,3295
4,FRX Sales Call,Detail with Sample,LINZESS,1,2510
5,FRX Sales Call,Detail with Sample,LINZESS,2,1894
6,IW Sales,Detail Only,LINZESS,1,1969
7,IW Sales,Detail with Sample,LINZESS,1,4770
8,IW Sales - HCO,Group Detail,LINZESS,1,81
9,IW Sales - HCO,Group Detail with Sample,LINZESS,1,58


#### Check for AHP Call topics

In [29]:
frequency_table = CALLS_ATT_PR.groupby(['CallFocus','ProductID']).size().reset_index(name='FREQUENCY')
frequency_table

,CallFocus,ProductID,FREQUENCY
0,,LINZESS,17
1,90 for 30,LINZESS,64
2,90 for 30 / Efficacy – Overall Abdom Symptoms,LINZESS,19
3,Efficacy – Overall Abdom Symptoms,LINZESS,303
4,Efficacy – Overall Abdom Symptoms / 90 for 30,LINZESS,24
5,Efficacy – Overall Abdom Symptoms / Efficacy –...,LINZESS,4
6,Efficacy – Overall Abdom Symptoms / OTC Patien...,LINZESS,4
7,Efficacy – Pain & Constipation,LINZESS,121
8,Efficacy – Pain & Constipation / Efficacy – Ov...,LINZESS,3
9,Efficacy – Pain & Constipation / Formulary,LINZESS,1


In [30]:
CALLS_ATT_PR_DISB_PROD = CALLS_ATT_PR_DISB1.copy()
CALLS_ATT_PR_DISB_PROD.rename(columns={'SalesRepID' : 'SalesRepIID'},inplace=True)


In [31]:
CALLS_ATT_PR_DISB_PROD.loc[(CALLS_ATT_PR_DISB_PROD['ProductID'] == 'AHP') & (CALLS_ATT_PR_DISB_PROD['CallProductDescription'] == ''), 'CallProductDescription'] = 'AHP'
CALLS_ATT_PR_DISB_PROD.loc[(CALLS_ATT_PR_DISB_PROD['ProductID'] == 'AHP/GIV') & (CALLS_ATT_PR_DISB_PROD['CallProductDescription'] == ''), 'CallProductDescription'] = 'AHP/GIV'


In [32]:
prod_dict = {
    'LINZESS': 'LIN',
    'COLOGUARD': 'CLG',
    'VIBERZI': 'VIB',
    'ZURAMPIC': 'ZUR',
    'DELZICOL/CANASA': 'DEC',
    'DUZALLO': 'DUZ',
    'AHP': 'AHP',
    'AHP/GIV': 'AHP/GIV'
}

CALLS_ATT_PR_DISB_PROD['PROD'] = CALLS_ATT_PR_DISB_PROD['ProductID'].map(prod_dict).fillna('ZZZ')

In [33]:
CALLS_ATT_PR_DISB_PROD['CallRecordType'] = CALLS_ATT_PR_DISB_PROD['CallRecordType'].str.strip()
src_dict = {
    'IW Sales': 'IRN',
    'IW Sales - HCO': 'IRN',
    'FRX Sales Call': 'FRX',
    'EXS Sales Call': 'PMR',
    'EXS MDR Call': 'MDR',
    'Engage Meeting': 'EGM',
    'ABBV ENGAGE MEETING': 'AEM',
    'Abbv Engage Meeting': 'AEM',
    'ALNY Sales Call': 'ALC',
    'Remote Call with Engage': 'REG',
    'Remote Call with FaceTime': 'RFT',
    'Remote Call with Other': 'ROT',
    'Remote Call with Teams': 'RTE',
    'Remote Call with Zoom': 'RZO',
    'Remote Call with Webex': 'RWE'
}
CALLS_ATT_PR_DISB_PROD['SRC'] = CALLS_ATT_PR_DISB_PROD['CallRecordType'].map(src_dict).fillna('XXX')

In [34]:
priority_dict = {
    1:'P',
    2:'S',
    3:'R'}

CALLS_ATT_PR_DISB_PROD['PRIORITY'] = CALLS_ATT_PR_DISB_PROD['ProductPriority'].map(priority_dict).fillna('X')

In [35]:
st_dict = {
    'DETAIL ONLY': 'CO',
    'DETAIL WITH SAMPLE': 'CS',
    'SAMPLE ONLY': 'SO',
    'PHONE': 'PH'
}

CALLS_ATT_PR_DISB_PROD['ST'] = CALLS_ATT_PR_DISB_PROD['CallType'].str.upper().map(st_dict).fillna('YY')

In [36]:
#MODIFY THIS TO ADD MORE OPTIONS
product_disb_dict = {
    '72 MCG' : '72',
    '145 MCG' : '145',
    '145 MCG SAMPLE 30CT' : 'S145',
    '290 MCG' : '290',
    '290 MCG SAMPLE 30CT' : 'S290'
}

CALLS_ATT_PR_DISB_PROD['PD'] = CALLS_ATT_PR_DISB_PROD['CallProductDescription'].str.upper().map(product_disb_dict).fillna('OOOO')

In [37]:
CALLS_ATT_PR_DISB_PROD['CALL_VARNM'] = CALLS_ATT_PR_DISB_PROD['SRC'] + CALLS_ATT_PR_DISB_PROD['PROD']\
+ CALLS_ATT_PR_DISB_PROD['PRIORITY'] + CALLS_ATT_PR_DISB_PROD['ST']

CALLS_ATT_PR_DISB_PROD.loc[CALLS_ATT_PR_DISB_PROD['CallProductCategory'].notna() & (CALLS_ATT_PR_DISB_PROD['CallProductCategory'] != 'Detail'), 'SAMPLE_VARNM'] \
= CALLS_ATT_PR_DISB_PROD['SRC'] + CALLS_ATT_PR_DISB_PROD['PD'] + 'X'

CALLS_ATT_PR_DISB_PROD['RECORD_DATE'] = CAL_RECORD_DATE

# Drop Redundant Columns
CALLS_ATT_PR_DISB_PROD.drop(columns={
'PROD','BrandReactionField','CallProductDisbursed','ProductDisbursedProductID','CallProductBatchLotNumber','PROD',
'SRC','PRIORITY','ST'},inplace=True)

In [38]:
DELTA_CALLS = CALLS_ATT_PR_DISB_PROD.drop_duplicates(subset = 'CallID')
callpweek_ALL_CALLS = pd.read_parquet(f's3://{bucket}/{callpweek}ALL_CALLS.parquet')

In [39]:
CALLS_OLD = pd.merge(
    callpweek_ALL_CALLS,DELTA_CALLS['CallID'],
    on = 'CallID',
    how = 'left',
    indicator = True
)

CALLS_OLD = CALLS_OLD[CALLS_OLD['_merge'] == 'left_only']
CALLS_OLD = CALLS_OLD.drop(['_merge'], axis=1).reset_index(drop=True)

ALL_CALLS = pd.concat([CALLS_ATT_PR_DISB_PROD, CALLS_OLD], ignore_index=True)
ALL_CALLS.sort_values(by=['CallID','AttendeeIID','DisbursementID'],inplace=True)

In [40]:
#Calls dataset is ready !  ( Note , All Calls and Active Calls dataset are no longer different)
ALL_CALLS.to_parquet(f's3://{bucket}/{calweek}ALL_CALLS.parquet')

In [41]:
## Add Source Check QC or any other QC here.
print(ALL_CALLS['CallDateTime'].max())
print(curwk_CALL['CallDateTime'].max())

2024-07-26 19:26:00
2024-07-26 19:26:00


#### 16 - CALLS - Weekly Building Final Data Sets

In [42]:
mask = (ALL_CALLS['CallDateTime'].dt.date >= CUT_OFF2) & (ALL_CALLS['CallDateTime'].dt.date < CUT_OFF)
CALLS_CUTOFF = ALL_CALLS.loc[mask]
# Please note that when using regular datetime[ns] variables to compare CallDateTime , I was getting more NOBS , but using this method makes it match with SAS

MAX_CALL_DATE = CALLS_CUTOFF['CallDateTime'].max()
MIN_CALL_DATE = CALLS_CUTOFF['CallDateTime'].min()
print(f'16.0.1 : Max CALLDATE\nThe latest value in the CALLDATE variable (cut_off): {MAX_CALL_DATE.strftime("%d-%b-%Y")}')
print(f'16.0.2 : Min CALLDATE\nThe oldest value in the CALLDATE variable (cut_off2): {MIN_CALL_DATE.strftime("%d-%b-%Y")}')

16.0.1 : Max CALLDATE
The latest value in the CALLDATE variable (cut_off): 12-Jul-2024
16.0.2 : Min CALLDATE
The oldest value in the CALLDATE variable (cut_off2): 09-Jul-2022


In [43]:
# Create data param dataset
DATE_PARM = pd.DataFrame()
DATE_PARM = pd.DataFrame(pd.date_range(start=MIN_CALL_DATE.date(), end=MAX_CALL_DATE.date()), columns=['DTE'])
DATE_PARM['DTE_shifted'] = DATE_PARM['DTE'] + pd.DateOffset(days=2)
DATE_PARM['MONTH'] = (DATE_PARM['DTE'].dt.to_period('M').dt.to_timestamp().rank(method='dense', ascending=False)).astype(int)
#DATE_PARM['WEEK'] = (DATE_PARM['DTE'].dt.to_period('W').dt.to_timestamp().rank(method='dense', ascending=False)).astype(int) # this method keps monday to sunday date
DATE_PARM['WEEK'] = (DATE_PARM['DTE_shifted'].dt.to_period('W').dt.to_timestamp().rank(method='dense', ascending=False)).astype(int)
DATE_PARM.drop(columns=['DTE_shifted'], inplace=True)

In [44]:
frequency_table = CALLS_CUTOFF.groupby(['CallRecordType', 'ProductPriority']).size().reset_index(name='FREQUENCY')
print("16.1: Check all combination: CALLRECORDTYPE * PRODUCTPRIORITY")
frequency_table

16.1: Check all combination: CALLRECORDTYPE * PRODUCTPRIORITY


,CallRecordType,ProductPriority,FREQUENCY
0,ABBV ENGAGE MEETING,1,10660
1,ABBV ENGAGE MEETING,2,2305
2,ABBV ENGAGE MEETING,3,2
3,Engage Meeting,1,113
4,FRX Sales Call,1,695970
5,FRX Sales Call,2,295323
6,FRX Sales Call,3,130
7,IW Sales,1,694467
8,IW Sales - HCO,1,3245
9,Remote Call with FaceTime,1,205


In [45]:
ALL_CALLS = CALLS_CUTOFF.copy()
ALL_CALLS['DTE'] = ALL_CALLS['CallDateTime'].dt.normalize()
ALL_CALLS = pd.merge(ALL_CALLS, DATE_PARM, on='DTE', how='left')
# ALL_CALLS.loc[ALL_CALLS['MONTH'].isna(), 'MONTH'] = None
# ALL_CALLS.loc[ALL_CALLS['WEEK'].isna(), 'WEEK'] = None 
# adding isna() check causes values to become float , leaving it commented for now
ALL_CALLS['CALL'] = 1

In [46]:
CALLS_DEDUPE = ALL_CALLS.drop_duplicates(subset = ['CallID','AttendeeIID','CALL_VARNM','CallDateTime'],ignore_index = True)
CALLS_DEDUPE.shape

(1256573, 28)

In [47]:
frequency_table = ALL_CALLS.groupby(['CallRecordType', 'CallType','ProductPriority','CALL_VARNM']).size().reset_index(name='FREQUENCY')
print("16.2: Check all combination: CALLRECORDTYPE * CALLTYPE * PRODUCTPRIORITY * CALL_VARNM")
frequency_table

16.2: Check all combination: CALLRECORDTYPE * CALLTYPE * PRODUCTPRIORITY * CALL_VARNM


,CallRecordType,CallType,ProductPriority,CALL_VARNM,FREQUENCY
0,ABBV ENGAGE MEETING,Detail Only,1,AEMLINPCO,10644
1,ABBV ENGAGE MEETING,Detail Only,2,AEMLINSCO,2292
2,ABBV ENGAGE MEETING,Detail Only,3,AEMLINRCO,2
3,ABBV ENGAGE MEETING,Detail with Sample,1,AEMLINPCS,16
4,ABBV ENGAGE MEETING,Detail with Sample,2,AEMLINSCS,13
5,Engage Meeting,Detail Only,1,EGMLINPCO,55
6,Engage Meeting,Detail with Sample,1,EGMLINPCS,58
7,FRX Sales Call,Detail Only,1,FRXLINPCO,506740
8,FRX Sales Call,Detail Only,2,FRXLINSCO,216798
9,FRX Sales Call,Detail Only,3,FRXLINRCO,130


In [48]:
frequency_table = ALL_CALLS.groupby(['ProductPriority']).size().reset_index(name='FREQUENCY')
print("16.3: Check all PRODUCTPRIORITY")
frequency_table

16.3: Check all PRODUCTPRIORITY


,ProductPriority,FREQUENCY
0,1,1404687
1,2,297628
2,3,132


In [49]:
print('16.3.1 : MAX_WEEK')
print(ALL_CALLS['WEEK'].max())
print('16.3.2 : MAX_MTH')
print(ALL_CALLS['MONTH'].max())


16.3.1 : MAX_WEEK
105
16.3.2 : MAX_MTH
25


In [50]:
frequency_table = CALLS_CUTOFF.groupby(['CALL_VARNM']).size().reset_index(name='FREQUENCY')
print("16.4: Check all CALL_VARNM and the number of CALL_VARNM")
frequency_table


16.4: Check all CALL_VARNM and the number of CALL_VARNM


,CALL_VARNM,FREQUENCY
0,AEMLINPCO,10644
1,AEMLINPCS,16
2,AEMLINRCO,2
3,AEMLINSCO,2292
4,AEMLINSCS,13
5,EGMLINPCO,55
6,EGMLINPCS,58
7,FRXLINPCO,506740
8,FRXLINPCS,189230
9,FRXLINRCO,130


In [51]:
print("16.4.1: Number of combinations (distinct CALL_VARNAME)")
print(len(CALLS_CUTOFF['CALL_VARNM'].unique()))

16.4.1: Number of combinations (distinct CALL_VARNAME)
23


##### CREATE_BUCKETS

In [52]:
CALLS_DEDUPE1 = CALLS_DEDUPE.sort_values(by = 'CALL_VARNM',ignore_index=True)
max_month = CALLS_DEDUPE1['MONTH'].max()

CALLS_DEDUPE2 = CALLS_DEDUPE1.copy()
max_month = CALLS_DEDUPE2['MONTH'].max()
all_combinations = pd.MultiIndex.from_product([CALLS_DEDUPE2['CALL_VARNM'].unique(), 
                                               np.arange(1, max_month + 1)], 
                                              names=['CALL_VARNM', 'MONTH']).to_frame(index=False)

CALLS_DEDUPE3 = pd.merge(all_combinations, CALLS_DEDUPE2, on=['CALL_VARNM', 'MONTH'], how='left')

CALLS_DEDUPE3['CALL'].fillna(0, inplace=True)

CALLS_DEDUPE3['CALL_MONTH'] = CALLS_DEDUPE3['CALL_VARNM'] + CALLS_DEDUPE3['MONTH'].astype(str)

CALLS_DEDUPE4 = pd.pivot_table(CALLS_DEDUPE3, 
                               values='CALL', 
                               index='AttendeeIID', 
                               columns='CALL_MONTH', 
                               aggfunc='sum')

CALLS_DEDUPE4 = CALLS_DEDUPE4.reindex(columns = CALLS_DEDUPE3['CALL_MONTH'].unique(), fill_value=0)
CALLS_DEDUPE4.fillna(0, inplace=True)
CALLS_DEDUPE4.reset_index(inplace=True)
CALLS_DEDUPE4.rename(columns={'AttendeeIID':'IID'},inplace = True)

#Reason for applyig reindex - 
#The pivot_table function in pandas does not include columns for missing combinations of the index and columns arguments. 
#In out case, if there are no records for a particular AttendeeIID and CALL_MONTH combination, that CALL_MONTH will not appear 
#as a column in the pivoted DataFrame for that AttendeeIID.

In [53]:
detail_cols = [col for col in CALLS_DEDUPE4.columns if (col.startswith('IRNLIN') or col.startswith('FRXLIN')) and 'CO' in col]
detail_with_sample_cols = [col for col in CALLS_DEDUPE4.columns if (col.startswith('IRNLIN') or col.startswith('FRXLIN')) and 'CS' in col]
sample_cols = [col for col in CALLS_DEDUPE4.columns if (col.startswith('IRNLIN') or col.startswith('FRXLIN')) and 'SO' in col]


CALLS_IID = CALLS_DEDUPE4.copy()
CALLS_IID_DETAIL = CALLS_IID[['IID'] + detail_cols]
CALLS_IID_DETAIL_WITH_SAMPLE = CALLS_IID[['IID'] + detail_with_sample_cols]
CALLS_IID_SAMPLE = CALLS_IID[['IID'] + sample_cols]

if (CALLS_IID_SAMPLE.shape[1]) == 1:
    print("No Sample Only Calls for IRN or FRX !")
    
# sdap to to storage location 
# sdap(calmnth,CALLS_IID)
# sdap(calmnth,CALLS_IID_DETAIL)
# sdap(calmnth,CALLS_IID_DETAIL_WITH_SAMPLE)
# sdap(calmnth,CALLS_IID_SAMPLE)

CALLS_IID.to_parquet(f's3://{bucket}/{calmnth}CALLS_IID.parquet')
CALLS_IID_DETAIL.to_parquet(f's3://{bucket}/{calmnth}CALLS_IID_DETAIL.parquet')
CALLS_IID_DETAIL_WITH_SAMPLE.to_parquet(f's3://{bucket}/{calmnth}CALLS_IID_DETAIL_WITH_SAMPLE.parquet')
CALLS_IID_SAMPLE.to_parquet(f's3://{bucket}/{calmnth}CALLS_IID_SAMPLE.parquet')

In [54]:
SAMPLES = CALLS_CUTOFF.copy()
SAMPLES = SAMPLES[~(SAMPLES['DisbursementID'].isna()) & ~(SAMPLES['CallProductCategory'] == 'Detail')]
SAMPLES.rename(columns = {'AttendeeIID':'IID'},inplace = True)
SAMPLES['DTE'] = SAMPLES['CallDateTime'].dt.normalize()
SAMPLES = pd.merge(SAMPLES, DATE_PARM, on='DTE', how='left')

In [55]:
mask = ~SAMPLES['SAMPLE_VARNM'].str.contains('OOO')
DISBURSEMENTS = SAMPLES[mask]
DISB_MISSING_CATEGORY = SAMPLES[~mask]

DISBURSEMENTS_SAMPLE = DISBURSEMENTS[DISBURSEMENTS['CallProductCategory'].isin(['Sample', 'Direct Ship'])]

In [56]:
frequency_table = DISBURSEMENTS_SAMPLE.groupby(['CallProductDescription']).size().reset_index(name='FREQUENCY')
print("16.5: Check all CALLPRODUCTDESCRIPTION")
frequency_table


16.5: Check all CALLPRODUCTDESCRIPTION


,CallProductDescription,FREQUENCY
0,145 mcg,227247
1,290 mcg,195754
2,72 mcg,190723


In [57]:
DISBURSEMENTS_SORT = DISBURSEMENTS_SAMPLE.drop_duplicates(subset=['IID','DisbursementID','SAMPLE_VARNM','CallDateTime'],ignore_index = True)
DISBURSEMENTS_PRM = DISBURSEMENTS_SORT[['PD','CallRecordType','SAMPLE_VARNM']].copy()
DISBURSEMENTS_PRM.drop_duplicates(subset=['PD','CallRecordType'],inplace=True,ignore_index = True)
VARLIST_MTH = pd.MultiIndex.from_product([DISBURSEMENTS_PRM['SAMPLE_VARNM'].unique(), 
                                               np.arange(1, max_month + 1)], 
                                              names=['SAMPLE_VARNM', 'MONTH']).to_frame(index=False)

DISBURSEMENTS_SORT_1 = DISBURSEMENTS_SORT.sort_values(by = 'SAMPLE_VARNM',ignore_index=True)
DISBURSEMENTS_SORT_2 = pd.merge(VARLIST_MTH, DISBURSEMENTS_SORT_1, on=['SAMPLE_VARNM', 'MONTH'], how='left')
DISBURSEMENTS_SORT_2['CallProductQuantity'].fillna(0, inplace=True)
DISBURSEMENTS_SORT_2['SAMP_MONTH'] = DISBURSEMENTS_SORT_2['SAMPLE_VARNM'] + DISBURSEMENTS_SORT_2['MONTH'].astype(str)

DISBURSEMENTS_SORT_3 = pd.pivot_table(DISBURSEMENTS_SORT_2, 
                               values='CallProductQuantity', 
                               index='IID', 
                               columns='SAMP_MONTH', 
                               aggfunc='sum')


DISBURSEMENTS_SORT_3 = DISBURSEMENTS_SORT_3.reindex(columns = DISBURSEMENTS_SORT_2['SAMP_MONTH'].unique(), fill_value=0)
DISBURSEMENTS_SORT_3.fillna(0, inplace=True)
DISBURSEMENTS_SORT_3.reset_index(inplace=True)

DISBURSEMENTS_IID = DISBURSEMENTS_SORT_3.copy()
#sdap(calmnth,DISBURSEMENTS_IID)
DISBURSEMENTS_IID.to_parquet(f's3://{bucket}/{calmnth}DISBURSEMENTS_IID.parquet')

In [58]:
calweek_DISBURSEMENT_TYPES = SAMPLES[['CallProductCategory','CallProductDescription','PD']].copy()

m1=calweek_DISBURSEMENT_TYPES['CallProductCategory'] != 'Detail'
m2=calweek_DISBURSEMENT_TYPES['CallProductCategory'].isna()

calweek_DISBURSEMENT_TYPES = calweek_DISBURSEMENT_TYPES[m1 & ~m2].copy()
calweek_DISBURSEMENT_TYPES.drop_duplicates(subset=['CallProductCategory','CallProductDescription','PD'],inplace=True)
calweek_DISBURSEMENT_TYPES.rename(columns={'PD':'VARCODE'},inplace=True)

DISBURSEMENT_TYPES = calweek_DISBURSEMENT_TYPES.copy()
# sdap(calweek,DISBURSEMENT_TYPES)
# sdap(calmnth,DISBURSEMENT_TYPES)

DISBURSEMENT_TYPES.to_parquet(f's3://{bucket}/{calweek}DISBURSEMENT_TYPES.parquet')
DISBURSEMENT_TYPES.to_parquet(f's3://{bucket}/{calmnth}DISBURSEMENT_TYPES.parquet')

In [59]:
#DISBURSEMENTS_IID - may not have all possible SAMPLE_VARNM because OOOO 
#gets excluded and New CallProductDiscriptions dont get asigned a PD

#Check inefficiencies
#Also Dev for 16a.